In [35]:
import pandas as pd
import numpy as np
import warnings
import sys
import json
from sklearn.metrics import precision_score, recall_score, f1_score 
import random
import re


def load_train_data(file):
    test_word_list = []
    test_tag_list = []
    test_index_list = []
    
    word_list = []
    tag_list = []
    index_list = []
    index = 1
    count = 629
    for line in file:
        if line != "\n":
            tmp = line.replace("\n","").split()
            word = tmp[0]
            tag = tmp[1]
            if count >= 0 :
                word_list.append(word)
                tag_list.append(tag)
                index_list.append(index)
            else:
                test_word_list.append(word)
                test_tag_list.append(tag)
                test_index_list.append(index)
            index = index + 1
            
        else:
            index = 1
            count = count - 1
            
    return index_list, word_list, tag_list, test_index_list, test_word_list,test_tag_list

with open("pos_fixed.txt", "r") as f:
    train_data = f.readlines()
test_index_lists, test_word_lists, test_tag_lists,train_index_lists, train_word_lists, train_tag_lists = load_train_data(train_data)
#print(train_word_lists)
#print(train_index_lists)

data_dict = {"index": train_index_lists, "word": train_word_lists, "tag" : train_tag_lists}
test_data_dict = {"index": test_index_lists, "word": test_word_lists, "tag" : test_tag_lists}
test_data=pd.DataFrame(data_dict)
train_data = pd.DataFrame(test_data_dict)
# print(test_data)
# print(train_data)

## Enlarge the training dataset

In [36]:
train_data_path_2 = "train"
train_data_2 = pd.read_table(train_data_path_2, header =None, names = ["index", "word", "tag"])
train_data_2 = train_data_2.dropna()

train_data_path_3 = "dev"
train_data_3 = pd.read_table(train_data_path_3, header =None, names = ["index", "word", "tag"])
train_data_3 = train_data_3.dropna()

In [37]:
train_data_df = [train_data,train_data_2, train_data_3]
train_data = pd.concat(train_data_df)

In [38]:
# train_data

In [39]:
train_data

,index,word,tag
0,1,@paulwalk,USR
1,2,It,PRP
2,3,'s,VBZ
3,4,the,DT
4,5,view,NN
...,...,...,...
131763,13,join,VB
131764,14,the,DT
131765,15,winning,VBG
131766,16,bidder,NN


# Create Vocab

In [40]:
occurrences = train_data.groupby("word")["index"].count().reset_index(name = "occurrences")
train_data = pd.merge(train_data, occurrences, how='left', on = "word")
# backup
backup_train_data_raw = train_data.copy()

In [41]:
def IsFloat(s):
    pattern = '^-?\d+\.?\d*$'  
    match = re.match(pattern, s)
    return match != None
  
def IsDigit(s):
    if s[1:len(s)-1].replace(",","").isdigit():
        return True
    elif IsFloat(s[1:len(s)-1]):
        return True
    else:
        return False

def contain_Digit(word):
    for char in word:
        if char.isdigit():
            return True
    return False

In [8]:
def deal_unk(train_data, threshold):
  # count_unk = train_data[train_data["occurrences"] < threshold].sum()["occurrences"]

    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: "http" in x)), "<unk_http>")

    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x[0] == "@")), "<unk_usr>")
    
    train_data["word"] = train_data["word"]\
                      .mask((train_data["occurrences"] < threshold)
                            & (train_data["word"].apply(lambda x: x.replace(",","").isdigit())), "<unk_num>")
    train_data["word"] = train_data["word"]\
                      .mask((train_data["occurrences"] < threshold)
                            & (train_data["word"].apply(lambda x: IsFloat(x))), "<unk_num>")
    
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x.isupper())), "<unk_upper>")
    
#     train_data["word"] = train_data["word"]\
#                       .mask((train_data["occurrences"] < threshold)
#                             & (train_data["word"].apply(lambda x: contain_Digit(x))), "<unk_num>")
  
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: "-" in x)), "<unk_jj>")
  
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                                & (train_data["word"].apply(lambda x: x.istitle())), "<unk_title>")
  # train_data["word"] = train_data["word"]\
  #                     .mask((train_data["occurrences"] < threshold)
  #                           & (train_data["word"].apply(lambda x: IsDigit(x))), "<unk_num>")
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x[len(x)-2:] == "ed")), "<unk_ed>")
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x[len(x)-3:] == "ing")), "<unk_ing>")
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x[len(x)-1:] == "s")), "<unk_s>")
 
    train_data["word"] = train_data["word"].mask( (train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x not in ["<unk_jj>", "<unk_title>", "<unk_s>", "<unk_upper>"
                                                                                              "<unk_usr>", "<unk_num>", "<unk_ed>", "<unk_ing>", "<unk_http>"])), "<unk>") 

In [42]:
def deal_unk(train_data, threshold):
  # count_unk = train_data[train_data["occurrences"] < threshold].sum()["occurrences"]
                        
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x[0] == "@")), "<unk_usr>")
    
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: "http" in x)), "<unk_http>")
    
    train_data["word"] = train_data["word"]\
                      .mask((train_data["occurrences"] < threshold)
                            & (train_data["word"].apply(lambda x: x.replace(",","").isdigit())), "<unk_num>")
    train_data["word"] = train_data["word"]\
                      .mask((train_data["occurrences"] < threshold)
                            & (train_data["word"].apply(lambda x: IsFloat(x))), "<unk_num>")
    train_data["word"] = train_data["word"]\
                      .mask((train_data["occurrences"] < threshold)
                            & (train_data["word"].apply(lambda x: contain_Digit(x))), "<unk_num>")
    
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x.isupper())), "<unk_upper>")
  
  
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: "-" in x)), "<unk_jj>")
  
   
  
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                                & (train_data["word"].apply(lambda x: x.istitle())), "<unk_title>")
  # train_data["word"] = train_data["word"]\
  #                     .mask((train_data["occurrences"] < threshold)
  #                           & (train_data["word"].apply(lambda x: IsDigit(x))), "<unk_num>")
    
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x[len(x)-2:] == "ed")), "<unk_ed>")
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x[len(x)-3:] == "ing")), "<unk_ing>")
    train_data["word"] = train_data["word"].mask((train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x[len(x)-1:] == "s")), "<unk_s>")
 
    train_data["word"] = train_data["word"].mask( (train_data["occurrences"] < threshold) 
                                              & (train_data["word"].apply(lambda x: x not in ["<unk_jj>", "<unk_title>", "<unk_s>", "<unk_upper>",
                                                                                              "<unk_usr>", "<unk_num>", "<unk_ed>", "<unk_ing>", "<unk_http>"])), "<unk>") 

In [43]:
threshold = 2
deal_unk(train_data, threshold) 

In [44]:
# backup
backup_train_data_unk = train_data.copy()

In [45]:
# train_data

In [46]:
special = train_data[train_data["occurrences"] < threshold][["word","occurrences"]]
special = special.groupby("word")["occurrences"].sum().reset_index(name = "occurrences")
special.sort_values("occurrences", inplace = True, ascending = False)

In [47]:
total_unk = sum(special["occurrences"])

In [48]:
train_data_remain = train_data[train_data["occurrences"] >= threshold][["word","occurrences"]]
train_data_remain.sort_values("occurrences", inplace = True, ascending = False)
train_data_remain = train_data_remain.drop_duplicates(keep="first")

In [49]:
vocabulary = pd.concat([special,train_data_remain ]).reset_index(drop=True)
vocabulary["index"] = vocabulary.index 

In [50]:
# Save to file
vocabulary[["word", "index", "occurrences"]].to_csv("vocab.txt", sep='\t', index=None, header = False)

In [51]:
# information about dictionary
# total size of vocabulary
print("The total size of the vocabulary is:", len(vocabulary))
#total occurrences of the special
print("The total occurrences of all types of the special token \'<unk>\' is:", total_unk)
# sperated information about <unk>

print("The total size of the \'<unk_usr>\' is:", int(special[special["word"] == "<unk_usr>"].occurrences))
print("The total size of the \'<unk_num>\' is:", int(special[special["word"] == "<unk_num>"].occurrences))
print("The total size of the \'<unk_ed>\' is:", int(special[special["word"] == "<unk_ed>"].occurrences))
print("The total size of the \'<unk_ing>\' is:", int(special[special["word"] == "<unk_ing>"].occurrences))
print("The total size of the \'<unk_s>\' is:", int(special[special["word"] == "<unk_s>"].occurrences))
print("The total size of the \'<unk_title>\' is:", int(special[special["word"] == "<unk_title>"].occurrences))
print("The total size of the \'<unk_jj>\' is:", int(special[special["word"] == "<unk_jj>"].occurrences))
print("The total size of the \'<unk_http>\' is:", int(special[special["word"] == "<unk_http>"].occurrences))
print("The total size of the \'<unk_upper>\' is:", int(special[special["word"] == "<unk_upper>"].occurrences))
print("The total size of the \'<unk>\' is:", int(special[special["word"] == "<unk>"].occurrences))

The total size of the vocabulary is: 25545
The total occurrences of all types of the special token '<unk>' is: 22985
The total size of the '<unk_usr>' is: 343
The total size of the '<unk_num>' is: 3607
The total size of the '<unk_ed>' is: 960
The total size of the '<unk_ing>' is: 849
The total size of the '<unk_s>' is: 2144
The total size of the '<unk_title>' is: 6058
The total size of the '<unk_jj>' is: 3291
The total size of the '<unk_http>' is: 126
The total size of the '<unk_upper>' is: 1118
The total size of the '<unk>' is: 4489


# Calculate transition and emssion

In [52]:
train_data = backup_train_data_unk.copy()

In [53]:
count_s = train_data.groupby(by=train_data["tag"])["index"].count().reset_index(name = "count_s")
count_s.columns=['s',"count_s"]

## Trainsition

In [54]:
# count(s-->s')
train_data = backup_train_data_unk.copy()
train_data["tag_2"] = train_data["tag"][1:].reset_index(drop=True)
train_data["tag_2_index"] = train_data["index"][1:].reset_index(drop=True)
train_data.loc[train_data["tag_2_index"] == 1.0, "tag_2"] = None
train_data["pair_t"] = train_data["tag"] + " " + train_data["tag_2"]

count_pair_t = train_data.groupby(by=train_data["pair_t"])["index"].count().reset_index(name = "count_pair_t").dropna()
count_pair_t["s"] = count_pair_t["pair_t"].apply(lambda x: x.split()[0])

# compute transition
transition = pd.merge(count_pair_t, count_s, how='left', on = "s")
transition["value"] = transition["count_pair_t"]/transition["count_s"]   

In [55]:
transition["pair"] = transition["pair_t"].apply(lambda x: str((str(x.split()[0]),str(x.split()[1]))))
transition = transition[["pair", "value"]]

In [56]:
transition = transition.set_index(["pair"])
transition_dic = transition.to_dict("dic")["value"]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


## emission

In [57]:
# count(s-->x)
train_data = backup_train_data_unk.copy()
train_data["pair_e"]= train_data["tag"] + " " + train_data["word"]

count_pair_e = train_data.groupby(by=train_data["pair_e"])["index"].count().reset_index(name = "count_pair_e").dropna()
count_pair_e["s"] = count_pair_e["pair_e"].apply(lambda x: x.split()[0])

# compute emission
emission = pd.merge(count_pair_e, count_s, how='left', on = "s")
emission["value"] = emission["count_pair_e"]/emission["count_s"]  # round(, 4)
emission["pair"] = emission["pair_e"].apply(lambda x: str((str(x.split()[0]),str(x.split()[1]))))
# emission["pair"] = emission["pair_e"].apply(lambda x: tuple([str(x.split()[0]),str(x.split()[1])]))
emission = emission[["pair", "value"]]

In [58]:
emission = emission.set_index(["pair"])
emission_dic = emission.to_dict("dic")["value"]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


## Save to json file

In [59]:
hmm_json = {}
hmm_json["trainsition"] = transition_dic
hmm_json["emission"] = emission_dic 

json_str = json.dumps(hmm_json)
with open("hmm.json", "w") as json_file:
    json_file.write(json_str)

In [60]:
# information about transition and emission
print("There are", len(transition), "transition in HMM.")
print("There are", len(emission), "emission in HMM.")

There are 1580 transition in HMM.
There are 33740 emission in HMM.


# Viterbi Decoding with HMM 

In [61]:
# compute first tag probability : t(s1)
first_word = backup_train_data_unk.copy()
first_word = first_word[first_word["index"] == 1]
sentence_num = len(first_word)
tag_count = first_word.groupby(by=train_data["tag"])["index"].count().reset_index(name = "count")
tag_count["t(s1)"] = tag_count["count"] / sentence_num
tag_count = tag_count[["tag", "t(s1)"] ].set_index(["tag"])
first_word_t = tag_count.to_dict("dic")["t(s1)"] 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [62]:
# possible tag dictionary
word_tag = backup_train_data_unk.copy()
def concat_func(x):
    return pd.Series({
        'tag': ' '.join(x['tag'].unique()).split(),
    })
word_tag = word_tag.groupby(word_tag["word"]).apply(concat_func).reset_index()
word_tag_dic = word_tag.set_index(["word"]).to_dict("dic")["tag"]

In [72]:
# word_tag_dic["<unk_upper>"]

In [ ]:
# tag dictionary
tag_data = backup_train_data_unk.copy()
tag_dic = tag_data.groupby(by=train_data["tag"])["index"].count().reset_index(name = "count")
tag_dic = tag_dic.set_index(["tag"])
tag_dic = tag_dic.to_dict("dic")["count"]

In [63]:
def viterbi_decoding(sentence, first_word_t, transition_dic, emission_dic, word_tag_dic):
    n = len(sentence)
    path = {}
    
    # first word
    probability = [{}]
    x1 = sentence[0]
    if x1 not in word_tag_dic:
        if "http" in x1:
            x1 = "<unk_http>"
        elif x1[0] == "@":
            x1 = "<unk_usr>"
        elif x1.replace(",","").isdigit() or IsFloat(x1) : #or contain_Digit(x1)
            x1 = "<unk_num>"
#         elif IsDigit(x1):
#             x1 = "<unk_num>"
        elif x1.isupper():
            x1 = "<unk_upper>"
        elif "-" in x1:
            x1 = "<unk_jj>"
        elif x1[len(x1)-2:] == "ed":
            x1 = "<unk_ed>"
        elif x1[len(x1)-3:] == "ing":
            x1 = "<unk_ing>"
        elif x1[len(x1)-1:] == "s":
            x1 = "<unk_s>"
        elif x1.istitle():
            x1 = "<unk_title>"
        elif IsFloat(x1):
            x1 = "<unk_num>"
        else:
            x1 = "<unk>"
    potential_tag_curr = word_tag_dic[x1]
    # print(potential_tag_curr)
    for state in potential_tag_curr:
        if state not in first_word_t:
            probability[0][state] = 0
        elif str((state, x1)) not in emission_dic:
            probability[0][state] = 0
        else:   
            probability[0][state] = first_word_t[state] * emission_dic[str((state, x1))]
        path[state] = [state]
    # print(cur_pro)
    # print(path)
    
    # HMM
    for i in range(1,n):
        xi = sentence[i]
        # print(xi)
        if xi not in word_tag_dic:
            if "http" in xi:
                xi = "<unk_http>"
            elif xi[0] == "@":
                xi = "<unk_usr>"
            elif xi.replace(",","").isdigit():
                xi = "<unk_num>"
            elif IsFloat(xi) : # or contain_Digit(xi)
                xi = "<unk_num>"
            elif xi.isupper():
                xi = "<unk_upper>"
            
#             elif IsDigit(xi):
#                 xi = "<unk_num>"
            elif "-" in xi:
                xi = "<unk_jj>"
            
            elif xi[len(xi)-2:] == "ed":
                xi = "<unk_ed>"
            elif xi[len(xi)-3:] == "ing":
                xi = "<unk_ing>"
            elif xi[len(xi)-1:] == "s":
                xi = "<unk_s>"
            elif xi.istitle():
                xi = "<unk_title>"
            else:
                xi = "<unk>"
        new_path = {}
        probability.append({}) 
        potential_tag_last = potential_tag_curr
        # print(xi)
        potential_tag_curr = word_tag_dic[xi]
        # print(potential_tag_curr)
        # print(potential_tag_last)

        record_pro = {}
        for state in potential_tag_curr:
            for s in potential_tag_last:
                if str((s,state)) not in transition_dic:
                    record_pro[s] = 0
                elif str((state, xi)) not in emission_dic:
                    record_pro[s] = 0
                else:
                    record_pro[s] = probability[i-1][s]*transition_dic[str((s,state))]*emission_dic[str((state, xi))]
            max_pro, last_state = max(zip(record_pro.values(), record_pro.keys()))
            # print(state)
            probability[i][state] = max_pro
            new_path[state] = path[last_state] + [state]
        path = new_path
    
    # find max probability
    # print(path)
    end_tag = potential_tag_curr
    max_pro, end_state = max([(probability[n - 1][end], end) for end in end_tag])
    return path[end_state]

## Prediction on Test dataset

In [64]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [65]:
test_data["next"] = test_data['index'][1:].reset_index(drop = True)
# test_data

In [66]:
test_data_acc = {}
sequence = ""
tag = ""
for index, row in test_data.iterrows():
    if row['next'] is None or row['next'] != 1:
        sequence = sequence + " " + row["word"]
        tag = tag + " " + row["tag"]
    else:
        sequence = sequence + " " + row["word"]
        tag = tag + " " + row["tag"]
        test_data_acc[sequence] = tag
        sequence = ""
        tag = ""

In [67]:
from sklearn.metrics import accuracy_score 
acc_viterbi = AverageMeter()

res = {}
for sentence, tags in test_data_acc.items():
    viterbi_pre = viterbi_decoding(sentence.split(), first_word_t, transition_dic, emission_dic, word_tag_dic)
    res[sentence] = (viterbi_pre, tags.split())
    acc = accuracy_score(tags.split(), viterbi_pre)
    acc_viterbi.update(acc) 

In [68]:
print("The accuracy of Viterbi Decoding on the dev data is {}%".format(acc_viterbi.avg * 100))

The accuracy of Viterbi Decoding on the dev data is 81.40570512937838%


In [69]:
n = len(res)
with open("viterbi.out", "w") as f:
    for key, value in res.items():
        sentence_list = key.split()
        for i in range(len(sentence_list)):
            f.write(str(i+1) + "\t" + sentence_list[i] + "\t" + value[1][i] +  "\t" + value[0][i] +"\n")
        n = n - 1
        if n != 0:
            f.write("\n") 